In [32]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import datetime as dt

In [9]:
def load_financial_data(name, output_file):
    try:
        df = pd.read_pickle(output_file)
        print('File data found...reading',name,'data')
    except FileNotFoundError:
        print('File not found...downloading', name, 'data')
        df = data.DataReader(name, 'yahoo', '2001-01-01', '2019-11-24')
        df.to_pickle(output_file)
    return df

In [10]:
SP500 = load_financial_data('^GSPC', '^GSPC_data.pkl')

File not found...downloading ^GSPC data


In [49]:
def get_logreturn(yahoo_dataframe):
    out = pd.DataFrame(index = yahoo_dataframe.index)
    prices = yahoo_dataframe.Close
    out['LogReturn'] = np.log(prices.shift(1, 'd')/prices)/np.sqrt( (out.index.shift(1, 'd')-out.index).days )
    return out

In [50]:
get_logreturn(SP500)

ValueError: operands could not be broadcast together with shapes (5786,) (4754,) 

In [13]:
SP500

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2001-01-02,1320.280029,1276.050049,1320.280029,1283.270020,1129400000,1283.270020
2001-01-03,1347.760010,1274.619995,1283.270020,1347.560059,1880700000,1347.560059
2001-01-04,1350.239990,1329.140015,1347.560059,1333.339966,2131000000,1333.339966
2001-01-05,1334.770020,1294.949951,1333.339966,1298.349976,1430800000,1298.349976
2001-01-08,1298.349976,1276.290039,1298.349976,1295.859985,1115500000,1295.859985
...,...,...,...,...,...,...
2019-11-18,3124.169922,3112.060059,3117.909912,3122.030029,3436690000,3122.030029
2019-11-19,3127.639893,3113.469971,3127.449951,3120.179932,3590070000,3120.179932
2019-11-20,3118.969971,3091.409912,3114.659912,3108.459961,4034890000,3108.459961
